In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from matplotlib.markers import MarkerStyle
from matplotlib.patches import Circle

In [ ]:
def zplane_plot(ax, z, p):
    """Realize a zplane plot.

    input:
    ax...axes handle
    z...zeros
    p...poles
    output:
    zplane plot into ax
    """
    if ax is None:
        ax = plt.gca()

    ax.axvline(0, color="0.7")
    ax.axhline(0, color="0.7")
    unit_circle = Circle((0, 0), radius=1, fill=False,
                         color="black", linestyle="-", alpha=0.9)
    ax.add_patch(unit_circle)
    
    ax.plot(np.real(z), np.imag(z),
            "o", label="zeros",
            color="C0", fillstyle="none",
            markersize=10, markeredgewidth=2)
    ax.plot(np.real(p), np.imag(p),
            "x", label="poles",
            color="C1", fillstyle="none",
            markersize=10, markeredgewidth=2)
    if (z[0]-z[1])==0.:
        ax.text(np.real(z[0])+0.05, np.imag(z[0])+0.05, '2', color='C0')     
    if (p[0]-p[1])==0.:
        ax.text(np.real(p[0])+0.05, np.imag(p[0])+0.05, '2', color='C1')    
    
    ax.set_xscale("linear")
    ax.set_yscale("linear")
    ax.set_xlabel(r'Real{$z$}', color="k")
    ax.set_ylabel(r'Imag{$z$}', color="k")
    ax.set_title("Poles x and zeros o of discrete-time domain filter",
                 color="k")
    ax.axis("equal")
    ax.set_xlim(-1.25, +1.25)
    ax.set_xticks(np.arange(-1.25, 1.25+0.25, 0.25))
    ax.set_xticklabels(["-1.25", "-1", "-0.75", "-0.5", "-0.25", "0",
                        "0.25", "0.5", "0.75", "1", "1.25"],
                       color="k")
    ax.set_ylim(-1.25, +1.25)
    ax.set_yticks(np.arange(-1.25, 1.25+0.25, 0.25))
    ax.set_yticklabels(["-1.25", "-1", "-0.75", "-0.5", "-0.25", "0",
                        "0.25", "0.5", "0.75", "1", "1.25"],
                       color="k")
    ax.legend(loc="best")
    ax.grid(True, which="both", axis="both",
            linestyle="-", linewidth=0.5, color=(0.8, 0.8, 0.8))


def bode_plot(b, a, N=2**10, fig=None):
    if fig is None:
        fig = plt.figure()
    p = np.roots(a)
    z = np.roots(b)
    W, Hd = signal.freqz(b, a, N, whole=True)
    if Hd[0] == 0:
        Hd[0] = 1e-15  # avoid zero at DC for plotting dB

    gs = fig.add_gridspec(2, 2)
    # magnitude
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(W/np.pi, np.abs(Hd), "C7",
             label=r'$|H(\Omega)|$)',
             linewidth=2)
    ax1.set_xlim(0, 2)
    ax1.set_xticks(np.arange(0, 9)/4)
    #ax1.set_xlabel(r'$\Omega / \pi$', color='k')
    ax1.set_ylabel(r'$|H|$', color='k')
    ax1.set_title("Magnitude response", color='k')
    ax1.grid(True, which="both", axis="both",
             linestyle="-", linewidth=0.5, color=(0.8, 0.8, 0.8))

    # phase
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.plot(W/np.pi, (np.angle(Hd)*180/np.pi), "C7",
             label=r'$\mathrm{angle}(H('r'\omega))$',
             linewidth=2)
    ax2.set_xlim(0, 2)
    ax2.set_xticks(np.arange(0, 9)/4)
    ax2.set_xlabel(r'$\Omega / \pi$', color='k')
    ax2.set_ylabel(r'$\angle(H)$ / deg', color='k')
    ax2.set_title("Phase response", color='k')
    ax2.grid(True, which="both", axis="both",
             linestyle="-", linewidth=0.5, color=(0.8, 0.8, 0.8))

    # zplane
    ax3 = fig.add_subplot(gs[:, 1])
    zplane_plot(ax3, z, p)

    print(p, z)

In [ ]:
a = [1, 0, 0]
figsize = (12, 7)

In [ ]:
b = [1, +1, 0]
bode_plot(b, a, N=2**10, fig=plt.figure(figsize=figsize))

In [ ]:
b = [1, -1, 0]
bode_plot(b, a, N=2**10, fig=plt.figure(figsize=figsize))

In [ ]:
b = [1, 0, +1]
bode_plot(b, a, N=2**10, fig=plt.figure(figsize=figsize))

In [ ]:
b = [1, 0, -1]
bode_plot(b, a, N=2**10, fig=plt.figure(figsize=figsize))

In [ ]:
b = [1, -2, 1]
bode_plot(b, a, N=2**10, fig=plt.figure(figsize=figsize))

In [ ]:
b = [1, +2, 1]
bode_plot(b, a, N=2**10, fig=plt.figure(figsize=figsize))